# MLOps end-to-end system on GCP

## MLOps fundamentals

<img src="_image\mlops_funda.webp">

**MLOps** is a set of principles and best practices that ensure efficient and reliable ML systems.

## Modularity: loosely coupled architecture

ML system should be build as a set of modulars (**Modularity** - loosely coupled), refers to developing system based on **independence components**, each one with its own specific function or task. This facilitates the scalability and reusability of solutions, as well as the flexible integration of these single components to build more complete systems.

Each step of pipeline should be build independence (Even if input or output as the same. Ex: training pipeline and inference pipeline). Each of module could be reused identical steps.

## Pipelines Orchestration and Automation

Pipeline ML là chuỗi các nhiệm vụ liên kết với nhau để tạo ra một quy trình làm việc hoàn chỉnh. Trong MLOps, cần có sự **tự động hóa** và **điều phối** pipeline này để chúng hoạt động một cách mượt mà.

Cụ thể:
- Pipeline có thể tự động trigger bởi 1 action hoặc 1 event hoặc 1 lịch cụ thể nào đó
- Data được điều phối một cách tự động trong code

## Continuous Integration and Delivery (CI/CD) pipelines

**CI/CD pipelines** help automate tasks that previously needed to be done manually. With GCP, using **Cloud Build** which is the serverless CI/CD platform of GCP, we have constructed several workflows that enhance functional particularities and optimize frequent procedures.

Hệ thống sử dụng **Cloud Build** để tự động tạo các phiên bản cập nhật của Docker image và triển khai lên các dịch vụ như Cloud Run hoặc Artifact Registry.

**1. Trainer Image CI/CD workflow**

Triggered by a push commit that involves changes in **trainer source code**, or **trainer dependencies**, or **trainer Dockerfile definition**

- (1) Automate build an updated version of trainer docker image
- (2) Then automate push image to Artifact Registry

`cloudbuild-trainer-image-workflow.yaml`
```yaml
steps:
- id: 'Clone Cloud Source Repository'
  name: 'gcr.io/cloud-builders/gcloud'
  args: ['source', 'repos', 'clone',
         '${_CLOUD_SOURCE_REPOSITORY}',
         '--project=${PROJECT_ID}'
         ] 
         
- id: 'Build updated Trainer Docker image'
  name: 'gcr.io/cloud-builders/docker'
  args: ['build',
         '-f', '${_DOCKERFILE_NAME}',
         '-t', '${_ARTIFACT_REGISTRY_REPO_LOCATION}-docker.pkg.dev/${PROJECT_ID}/${_ARTIFACT_REGISTRY_REPO_NAME}/${_TRAINER_IMAGE_NAME}:${_TRAINER_IMAGE_TAG}',
         '.'
        ]
  # workdir to execute this step from
  dir: '${_CLOUD_SOURCE_REPOSITORY}/training-pipeline'

substitutions:
    _CLOUD_SOURCE_REPOSITORY: 'poc-mlops-asset'
    _CLOUD_SOURCE_REPOSITORY_URI: 'workspace/${_CLOUD_SOURCE_REPOSITORY}'
    _ARTIFACT_REGISTRY_REPO_LOCATION: 'us-central1'
    _ARTIFACT_REGISTRY_REPO_NAME: 'poc-mlops-asset'
    _TRAINER_IMAGE_NAME: 'time-series-trainer'
    _TRAINER_IMAGE_TAG: 'latest'
    _DOCKERFILE_NAME: 'trainer.Dockerfile'                          

options:
    dynamicSubstitutions: true
    logging: CLOUD_LOGGING_ONLY

tags: ['training-pipeline']

# This automatically pushes the built image to Artifact Registry
images: ['${_ARTIFACT_REGISTRY_REPO_LOCATION}-docker.pkg.dev/${PROJECT_ID}/${_ARTIFACT_REGISTRY_REPO_NAME}/${_TRAINER_IMAGE_NAME}:${_TRAINER_IMAGE_TAG}']
```

**2. Dashboard UI continuous deployment workflow**

Triggered by a pushed commit that involves changes in **customized Grafana instance Dockerfile**, **predefined dashboard and datasources configuration**, or **provisioned dashboards**
- (1) builds an updated version of the Dashboard UI Docker image, 
- (2) pushes it to our Artifact Registry
- (3) seamlessly deploys a new Cloud Run revision based on this updated version.


**3. (Training / Inference) YAML pipeline templates workflow**

Triggered by a pushed commit that includes a new compiled version of a pipeline template in the pipeline templates directories

- (1) extracts the pipeline version using regex from the filename
- (2) tags the template file with the version
- (3) pushes it to our associated Kubeflow Pipelines Artifact Registry.


## Experiment tracking

**Experiment tracking** giúp lưu trữ và so sánh các lần chạy pipeline, giúp khám phá hiệu suất của các mô hình với các tham số khác nhau. **Vertex AI Experiments** được sử dụng để theo dõi các thông số và kết quả của các lần chạy huấn luyện để phân tích hiệu suất.

<img src= "_image\tracking.webp">

In [ ]:
## task.py snippet from Trainer source code
aiplatform.init(project=PROJECT_ID, experiment=EXPERIMENT_NAME)
    with aiplatform.start_run(PIPELINE_RUN_NAME):
        # Log training params
        aiplatform.log_params(xgb_params)
        # ...
        # Log training performance (TensorBoard)
        for i, rmse in enumerate(val_error):
            aiplatform.log_time_series_metrics({'rmse': rmse}, step=i)
        # ...
        # Log training metrics
        aiplatform.log_metrics({"mse_train": mse_train,
                                "mse_test": mse_test})

## Model Registry

Easier **model management** and **versioning**, **acts as a repository** for ML models, and it represents the connection between our training (which registers the model) and inference (which retrieves the model) pipelines.

Vertex AI Model Registry được sử dụng để lưu trữ các mô hình dự báo, và các artifacts của mô hình được lưu trong Google Cloud Storage (GCS).

## Testing

Pipeline components should be designed so that they can be tested using **unit tests**. Hệ thống sử dụng **pytest** để kiểm tra các thành phần pipeline ML, đôi khi cần mô phỏng (mock) dữ liệu đầu vào và đầu ra khi kiểm thử.

## ML Metadata and Logging

Việc quản lý vòng đời của metadata từ các quy trình ML rất quan trọng để hỗ trợ gỡ lỗi khi có vấn đề. Dịch vụ **Vertex AI Metadata** và **Google Cloud Storage (GCS)**, **Logs Explorer** được dùng để lưu trữ các logs và metadata của các pipeline.

## Continuous monitoring

Build hệ thống monitoring

Tracking:
- data value and concept drift
- label drift
- model performance

<img src= "_image\monitoring.webp">

tham khảo platform: https://www.evidentlyai.com/pricing

## Versioning and reproducibility

- **Versioning** không chỉ áp dụng cho **source code** mà còn cho cả các **pipeline**, **docker image**, **model** và **train data**. Việc này giúp quản lý toàn bộ chu kỳ phát triển ML và hỗ trợ quá trình gỡ lỗi khi có sự cố.
Các pipeline và mô hình được lưu trữ và phiên bản hóa trong Artifact Registry, trong khi Vertex AI Datasets được sử dụng để phiên bản hóa các tập dữ liệu huấn luyện.

In GCP:
- **Pipeline templates** (both training and inference) and **trainer Docker images** are versioned and stored using a **Kubeflow Pipelines repository** and a **Docker repository** in **Artifact Registry**. 
- **ML models** versioning in **Vertex AI Model Registry**. 
- **Vertex AI Datasets** is in charge of versioning **training datasets**

## Use-case

Detailed overview of our newly designed MLOps forecasting system, which covers the entire data science lifecycle (DSLC), from data collection and training a model to enabling batch inference and monitoring model performance over time.

<img src="_image\ml_wf.gif">

---

**Used GCP services**

<div style="display: flex; align-items: center;">

<div style="flex: 1;">
  
- `BigQuery`: for time-series data storage, predictions storage and training/inference processed datasets storage
- `Vertex AI`: Vertex AI Pipelines (training and inference pipelines), Vertex AI Experiments (comparing training runs), Vertex AI Model Registry (versioning and storing forecasting models), Vertex AI Datasets (versioning and storing datasets), Vertex AI Metadata (pipelines metadata)
- `Artifact Registry`: registry and versioning of Docker images (Trainer and Cloud Run Dashboard UI) and Kubeflow Pipelines templates (training/inference)
- `Google Cloud Storage`: for pipeline artifacts storage, model artifacts storage, exploratory data analysis reports storage
- `Cloud Scheduler`: scheduled data collection and functions with specific purpose (e.g., generating daily monitoring reports)
- `Cloud Build`: CI/CD pipelines and other workflows
- `Cloud Run Functions`: for hosting our visualization dashboard (based on Grafana)

</div>

<div style="flex: 1; text-align: right;">
  
  <img src="_image\gcp_service.webp" alt="Description"/>
  
</div>

</div>


---

**Monorepo-based strategy**

All built modules are hosted in a single source code repository. This has allowed us to collaborate smoothly, manage the entire project modules in a centralized way and share codebases easier

Several support tools:
- [DevContainers](https://code.visualstudio.com/docs/devcontainers/containers) for module development environments management
- [Poetry](https://python-poetry.org/) for dependency management (and packaging)
- [Cloud Build](https://cloud.google.com/build?hl=en) for tasks automation as CI/CD pipelines
- [just](https://just.systems/) for quick, useful shortcuts while developing and testing